В отличии от друг *RTSD-R_MERGED.ipynb*, этот ноутбук нацелен на формировние обуч. датасета с учетом след. условий:

* Объеденить все знаки ограничения скорости
* Объеденить все знаки конца ограничений скорости

* Выплевывать датасет со всеми доступными знаками, т.к. он будет нужен для энкодера **иншааллах**!

In [ ]:
import pandas as pd
import os
import pathlib
import shutil
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

RTSD_PUBLIC_ROOT = pathlib.Path('D:\\Downloads\\rtsd-public\\classification')

PATH_TO_RTDS_R1 = RTSD_PUBLIC_ROOT / 'rtsd-r1.tar/rtsd-r1'
PATH_TO_RTDS_R3 = RTSD_PUBLIC_ROOT / 'rtsd-r3.tar/rtsd-r3'

PROJECT_ROOT = pathlib.Path(os.path.join(os.curdir, os.pardir))

DATA_DIR = PROJECT_ROOT / 'data'
NOTEBOOKS_DIR = PROJECT_ROOT / 'notebooks'

RTSD_MERGE_OUT_DIR = DATA_DIR / 'RTSD_CLASSIFIER_DATASET'
RTSD_MERGE_OUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
RTDS_R1_N2C = pd.read_csv(PATH_TO_RTDS_R1 / 'numbers_to_classes.csv')
RTDS_R1_N2C_DICT = RTDS_R1_N2C.set_index('class_number').to_dict()['sign_class']
display(RTDS_R1_N2C.head())

RTDS_R3_N2C = pd.read_csv(PATH_TO_RTDS_R3 / 'numbers_to_classes.csv')
RTDS_R3_N2C_DICT = RTDS_R3_N2C.set_index('class_number').to_dict()['sign_class']
display(RTDS_R3_N2C.head())

R1_TEST = pd.read_csv(PATH_TO_RTDS_R1 / 'gt_test.csv')
R1_TRAIN = pd.read_csv(PATH_TO_RTDS_R1 / 'gt_train.csv')
R3_TEST = pd.read_csv(PATH_TO_RTDS_R3 / 'gt_test.csv')
R3_TRAIN = pd.read_csv(PATH_TO_RTDS_R3 / 'gt_train.csv')


R1_TRAIN['class_number'] = R1_TRAIN['class_number'].apply(lambda x: RTDS_R1_N2C_DICT[x])
R1_TRAIN['filename'] = R1_TRAIN['filename'].apply(lambda x: 'rtsd-r1/train/' + x)

R1_TEST['class_number'] = R1_TEST['class_number'].apply(lambda x: RTDS_R1_N2C_DICT[x])
R1_TEST['filename'] = R1_TEST['filename'].apply(lambda x: 'rtsd-r1/test/' + x)

R3_TRAIN['class_number'] = R3_TRAIN['class_number'].apply(lambda x: RTDS_R3_N2C_DICT[x])
R3_TRAIN['filename'] = R3_TRAIN['filename'].apply(lambda x: 'rtsd-r3/train/' + x)

R3_TEST['class_number'] = R3_TEST['class_number'].apply(lambda x: RTDS_R3_N2C_DICT[x])
R3_TEST['filename'] = R3_TEST['filename'].apply(lambda x: 'rtsd-r3/test/' + x)

In [ ]:
R_MERGED = pd.concat(
    [
        R1_TRAIN, R3_TRAIN,
        R1_TEST, R3_TEST
    ],
    ignore_index=True
)
R_MERGED.rename({'filename': 'filepath', 'class_number': 'sign'}, axis=1, inplace=True)

In [ ]:
_, ax = plt.subplots(nrows=1, ncols=1, figsize=(21, 7))
ax.tick_params(labelrotation=90)
g = sns.countplot(
    ax=ax, 
    x='sign', 
    data=R_MERGED, 
    order=sorted(
        R_MERGED['sign'].value_counts().index.tolist()
    )
)
plt.tight_layout()

In [ ]:
R_MERGED['sign'] = R_MERGED['sign'].apply(lambda x: ".".join(x.split('_')[:-1]) if 'n' in x else x.replace('_', '.')) 

In [ ]:
_, ax = plt.subplots(nrows=1, ncols=1, figsize=(21, 7))
ax.tick_params(labelrotation=90)
g = sns.countplot(
    ax=ax, 
    x='sign', 
    data=R_MERGED, 
    order=sorted(
        R_MERGED['sign'].value_counts().index.tolist()
    )
)
plt.tight_layout()

Напомню кого не хватает

| Знак | Описание | Источник |
| ------------- | ------------- | ---- |
| 1.6 | Пересечение равнозначных дорог | - |
| 1.31 | Туннель | - |
| 2.4 | Уступите дорогу | GTSRB Recognition |
| 3.21 | Конец запрещения обгона | GTSRB Recognition |
| 3.22 | Обгон грузовым автомобилям запрещен | GTSRB Recognition |
| 3.23 | Конец запрещения обгона грузовым автомобилям | GTSRB Recognition |
| 3.24-90 | Огр 90 | - |
| 3.24-100 | Огр 100 | GTSRB Recognition |
| 3.24-110 | Огр 110 | - |
| 3.24-120 | Огр 120 | GTSRB Recognition |
| 3.24-130 | Огр 130 | - |
| 3.25 | Конец огр. максимальной скорости | GTSRB Recognition |
| 3.31 | Конец всех ограничений | GTSRB Recognition |
| 6.3.2 | Зона для разворота | - |

Что делать

| Знак | Описание | Источник |
| ------------- | ------------- | ---- |
| 1.6 | Пересечение равнозначных дорог | Надеемся на удачу |
| 1.31 | Туннель | Надеемся на удачу |
| 2.4 | Уступите дорогу | GTSRB Recognition |
| 3.21 | Конец запрещения обгона | GTSRB Recognition |
| 3.22 | Обгон грузовым автомобилям запрещен | GTSRB Recognition |
| 3.23 | Конец запрещения обгона грузовым автомобилям | GTSRB Recognition |
| 3.24-90 | Огр 90 | Объеденили |
| 3.24-100 | Огр 100 | Объеденили |
| 3.24-110 | Огр 110 | Объеденили |
| 3.24-120 | Огр 120 | Объеденили |
| 3.24-130 | Огр 130 | Объеденили |
| 3.25 | Конец огр. максимальной скорости | GTSRB Recognition |
| 3.31 | Конец всех ограничений | GTSRB Recognition |
| 6.3.2 | Зона для разворота | Надеемся на удачу |

Объединение уже сделали, дергаем из GTSRB 2.4, 3.21, 3.22, 3.23, 3.25, 3.31

In [ ]:
new_folders = ['2.4', '3.21', '3.22', '3.23', '3.25', '3.31']
sub_dfs = []
for folder in new_folders:
    temp_df = pd.DataFrame(os.listdir(RTSD_PUBLIC_ROOT / folder), columns=['filepath'])
    temp_df['filepath'] = temp_df['filepath'].apply(lambda x: folder + '/' + x)
    temp_df['sign'] = folder
    sub_dfs.append(temp_df)

In [ ]:
merged_sub_dfs = pd.concat(sub_dfs, ignore_index=True)
R_MERGED = pd.concat([R_MERGED, merged_sub_dfs], ignore_index=True)

In [ ]:
_, ax = plt.subplots(nrows=1, ncols=1, figsize=(21, 7))
ax.tick_params(labelrotation=90)
g = sns.countplot(
    ax=ax, 
    x='sign', 
    data=R_MERGED, 
    order=sorted(
        R_MERGED['sign'].value_counts().index.tolist()
    )
)
plt.tight_layout()

Undersampling + Oversampling

In [ ]:
R_MERGED_GROUPED = R_MERGED.groupby('sign', axis=0)
MEAN_BY_GROUPS = int(np.floor(R_MERGED_GROUPED.size().mean())) * 4
R_MERGED['set'] = np.nan
print(MEAN_BY_GROUPS)

In [ ]:
import random

SET_COLUMN_INDEX = R_MERGED.columns.get_loc("set")

for key, items in R_MERGED_GROUPED.groups.items():
        # print(items)
        items = list(items)     # явно приведем к списку для душевного спокойствия
        random.shuffle(items)   # перемешаем

        # print(key)

        if len(items) > MEAN_BY_GROUPS:
            # выбираем рандомные значения из этой группы в колличестве MEAN_BY_GROUPS*0.6 для train
            # MEAN_BY_GROUPS*0.2 для valid, остальное кинем в test
            # print(int(MEAN_BY_GROUPS*0.8))
            
            TEMP_ITEMS_INCLUDED = items[0:MEAN_BY_GROUPS]
            TEMP_ITEMS_EXCLUDED = items[MEAN_BY_GROUPS::]
            
            TRAIN_GROUP, VALID_GROUP, TEST_GPOUP = np.split(
                TEMP_ITEMS_INCLUDED, 
                [int(len(TEMP_ITEMS_INCLUDED)*0.6), 
                 int(len(TEMP_ITEMS_INCLUDED)*0.8)]
            )
            
            TEST_GPOUP = np.append(TEST_GPOUP, TEMP_ITEMS_EXCLUDED)

        else:
            TRAIN_GROUP, VALID_GROUP, TEST_GPOUP = np.split(items, [int(len(items)*0.6), int(len(items)*0.8)])
        
        R_MERGED.iloc[TRAIN_GROUP, SET_COLUMN_INDEX] = 'train'
        R_MERGED.iloc[VALID_GROUP, SET_COLUMN_INDEX] = 'valid'
        R_MERGED.iloc[TEST_GPOUP, SET_COLUMN_INDEX] = 'test'

_, ax = plt.subplots(nrows=3, ncols=1, figsize=(21, 14))

LABELS = ['train', 'valid', 'test']
for i in range(len(LABELS)):
    g = sns.countplot(x='sign', 
                      data=R_MERGED[R_MERGED['set']==LABELS[i]],  
                      ax=ax[i], 
                      order=sorted(R_MERGED['sign'].value_counts().index.tolist())
                     )
    ax[i].tick_params(labelrotation=90)
    ax[i].set_title(LABELS[i])
    plt.tight_layout()

In [ ]:
R_MERGED

In [ ]:
R_MERGED_TRAIN = R_MERGED[R_MERGED['set']=='train']
R_MERGED_TRAIN_GROUPED = R_MERGED_TRAIN.groupby('sign', axis=0)

for key, items in R_MERGED_TRAIN_GROUPED.groups.items():

    items = list(items)     # явно приведем к списку для душевного спокойствия
    random.shuffle(items)   # перемешаем

    if len(items) < MEAN_BY_GROUPS:
        ROWS_TO_FILL_COUNT = int(MEAN_BY_GROUPS*0.6) - len(items)
        ROWS_TO_APPEND = R_MERGED.iloc[items].sample(ROWS_TO_FILL_COUNT, replace=True)
        # print(ROWS_TO_APPEND)
        #print(len(gt.index))
        R_MERGED = R_MERGED.append(ROWS_TO_APPEND, ignore_index=True)
        #print(len(gt.index))
        
_, ax = plt.subplots(nrows=3, ncols=1, figsize=(21, 14))

LABELS = ['train', 'valid', 'test']
for i in range(len(LABELS)):
    g = sns.countplot(x='sign', 
                      data=R_MERGED[R_MERGED['set']==LABELS[i]],  
                      ax=ax[i], 
                      order=sorted(R_MERGED['sign'].value_counts().index.tolist())
                     )
    ax[i].tick_params(labelrotation=90)
    ax[i].set_title(LABELS[i])
    plt.tight_layout()

In [ ]:
R_MERGED

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

R_MERGED['encoded'] = le.fit_transform(R_MERGED['sign'])

In [ ]:
R_MERGED.to_csv('WIDE_DATASET_4_ENCODER.csv', index=False)
